# E2B Code Sandbox with Google Gemini
Execute AI-generated Python code in a secure sandbox environment.

## Setup and Imports

In [1]:
"""E2B Code Sandbox with Google Gemini."""
import warnings
warnings.filterwarnings('ignore')

import sys
import os
import base64
import pandas as pd

root_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd())))
sys.path.append(root_dir)

from dotenv import load_dotenv
load_dotenv()

from e2b_code_interpreter import Sandbox
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain.agents import create_agent
from langchain.messages import HumanMessage

## Initialize Model and Sandbox

In [2]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    convert_system_message_to_human=True
)

sbx = Sandbox.create()
print("Sandbox created")

Sandbox created


## Helper Functions

In [3]:
def get_dataset_info(file_path):
    """Get basic dataset info."""
    if file_path.endswith('.csv'):
        df = pd.read_csv(file_path, nrows=3)
    else:
        df = pd.read_excel(file_path, nrows=3)

    return f"Columns: {list(df.columns)}\nSample data:\n{df.to_string()}"

## Define Tool

In [4]:
@tool
def run_python_code(code: str):
    """Execute Python code in E2B sandbox and save chart outputs.

    Args:
        code: Python code to execute

    Returns:
        Execution result or error message
    """
    print('Running code in sandbox....')
    execution = sbx.run_code(code)
    print('Code execution finished!')

    if execution.error:
        return f"Error: {execution.error.name}\nValue: {execution.error.value}"

    results = []
    for idx, result in enumerate(execution.results):
        if result.png:
            filename = f'chart-{idx}.png'
            with open(filename, 'wb') as f:
                f.write(base64.b64decode(result.png))
            results.append(f'Chart saved to {filename}')

    return "\n".join(results) if results else "Code executed successfully"

## Upload Dataset and Run Analysis

In [5]:
data_file = "./data/IMDB-Movie-Data.csv"
query = "Create a line chart showing average ratings over years"

filename = os.path.basename(data_file)
with open(data_file, "rb") as f:
    sandbox_file = sbx.files.write(f"data/{filename}", f)

print(f"Uploaded: {sandbox_file.path}")

Uploaded: /home/user/data/IMDB-Movie-Data.csv


## Create Agent and Execute Query

In [6]:
dataset_info = get_dataset_info(data_file)

system_prompt = f"""You are a Python code generator for data analysis.

Dataset location: {sandbox_file.path}
{dataset_info}

CRITICAL RULES:
1. Generate ONLY executable Python code - no explanations, no markdown, no comments outside code
2. Import all required libraries (pandas, matplotlib, numpy, etc.)
3. Load data from the dataset path provided
4. For plots, MUST end with: display(plt.gcf())
5. Use the run_python_code tool with ONLY the code string

Example format:
```python
import pandas as pd
import matplotlib.pyplot as plt
df = pd.read_csv('{sandbox_file.path}')
# your analysis code
plt.figure(figsize=(10,6))
# your plotting code
display(plt.gcf())
```"""

agent = create_agent(
    model=model,
    tools=[run_python_code],
    system_prompt=system_prompt
)

result = agent.invoke({"messages": [HumanMessage(content=query)]})

response = result['messages'][-1].text
print(f"\nResponse:\n{response}")

Running code in sandbox....
Code execution finished!

Response:

